# YARN

Deze notebook verkent de mogelijkheden van het configureren van de Resource Manager binnen Hadoop.
Dit gaan we verkennen in het uitvoeren van de volgende stappen:
* Bestuderen van de Yarn-UI
* Beperken van de beschikbare cpu-cores en ram geheugen van de datanode
* Aanmaken van een extra resource type voor het aantal beschikbare GPU en GPU-memory (VRAM)
* Aangeven dat de single node van onze cluster beschikt over 1 GPU met 4 GB VRAM
* Aanmaken van een resource profile met 1 GPU en 2 GB VRAM

## Yarn UI

De UI geeft een reeks informatie over de beschikbare nodes in de cluster, de capabiliteiten van deze nodes en de applicatie die gescheduled, werkende, gefaald of geslaagd zijn.
De standaardlink voor deze UI te bekijken is: [http://localhost:8088](http://localhost:8088)
Deze ziet er uit als volgt:

![Yarn - homepage](images/ui_001.png)

waarbij de pagina met de informatie over de nodes eruit ziet als volgt:

![Yarn - node capabilities](images/ui_002.png)

## Beperken aantal cpu's en ram

De staardinstellingen voor het aantal cores en ram-geheugen dat beschikbaar is op een node is respectievelijk 8 cores en 8 GB Ram.
Dit is te veel voor de toepassingen die we momenteel gaan gebruiken en kan ook vaak niet gehaald worden door een laptop.
Om deze reden gaan we eerst dit beperken zodat we de laptop niet overbelasten.

Dit kan zowel in de yarn-site.xml of in de node-resources.xml file gedaan worden door het volgende toe te voegen.

    <property>
		  <description>Max available memory on each data node.</description>
		  <name>yarn.nodemanager.resource.memory-mb</name>
		  <value>2048</value>
	</property>

	<property>
		<description>Max available cores data node.</description>
		<name>yarn.nodemanager.resource.cpu-vcores</name>
		<value>4</value>
	</property>

**Welke waarden voor aantal cores en beschikbaar geheugen worden ingesteld door het bovenstaande te kiezen?**

<details>
  <summary>Klik hier voor het antwoord</summary>

  ![Yarn UI limited](images/limited.png)
</details>

Indien dit nog te veel is voor jouw PC/laptop, pas het nu aan naar een zelfgekozen waarde. Indien je de aanpassingen niet ziet verschijnen in de ui moet je waarschijnlijk nog de yarn restarten. Controleer met het commando jps of yarn correct opgestart is.

## Extra Resource Types

Standaard kan enkel het aantal cores en beschikbare ram-geheugen van een node bepaald worden.
Indien we ook de hoeveelheid VRAM dat beschikbaar is op een node willen geven aan de resource manager voor een optimale scheduling uit te voeren moeten we dit type resource nog aanmaken.
Het aanmaken van een resource type kan gedaan worden in yarn-site.xml of resource-types.xml door het volgende toe te voegen aan bijvoorbeeld resource-types.xml:

    <property>
        <name>yarn.resource-types</name>
        <value>vram</value>
    </property>
    <property>
        <name>yarn.resource-types.vram.units</name>
        <value>G</value>
    </property>

    <property>
        <name>yarn.resource-types.vram.minimum-allocation</name>
        <value>0</value>
    </property>

    <property>
        <name>yarn.resource-types.vram.maximum-allocation</name>
        <value>1024</value>
    </property>

**Let op:** Indien de file resource-types.xml nog niet bestaat kan je hem toevoegen door een andere xml te kopieren en alles te verwijderen behalve de configuration tags. Het bovenstaande stukje code moet tussen de configuration tags staan.

In bovenstaande stukje xml worden er drie zaken aangepast, namelijk:
* yarn.resource-types: Dit is een csv lijst met alle extra resources. In dit geval is dit enk "vram".
* yarn.resource-types.vram.units: Hier passen we de grootteorde aan van de waarden die we gebruiken voor het type vram (dit kan je doen per type via de naam van de variabele). De variabele G staat voor G of dus Gigabyte en andere opties kan je [hier](https://hadoop.apache.org/docs/stable/hadoop-yarn/hadoop-yarn-site/ResourceModel.html) vinden.
* yarn.resource-types.vram.minimum-allocation: De minimale hoeveelheid die kan toegekend worden aan een container voor het uitvoeren van een applicatie
* yarn.resource-types.vram.maximum-allocation: De maximale hoeveelheid die kan toegekend worden aan een enkele container.

Merk op dat je ook de Default Resource Calculator moet aanpassen. Dit kan gedaan worden in de capacity-scheduler.xml door de volgende waarde aan te passen van

    <property>
		  <name>yarn.scheduler.capacity.resource-calculator</name>
		  <value>org.apache.hadoop.yarn.util.resource.DefaultResourceCalculator</value>
	</property>

naar

    <property>
		  <name>yarn.scheduler.capacity.resource-calculator</name>
		  <value>org.apache.hadoop.yarn.util.resource.DominantResourceCalculator</value>
	</property>

Na het toevoegen van dit resource type moet dit ook zichtbaar zijn in de UI, bijvoorbeeld:

![Yarn UI with extra type](images/extraType.png)

Voeg nu als oefening een extra resource-type toe voor het aantal gpu's beschikbaar in een node. Dit wordt uitgedrukt in eenheden en heeft een minimum waarde van 0 en een maximum waarde van 4.

## Aangeven van de mogelijkheden van de datanode

In de vorige sectie hebben we twee nieuwe parameters toegevoegd om het mogelijk te maken van Yarn om rekening te houden met de grafische kaarten aanwezig in de verschillende nodes van de cluster en een minimum of maximum aantal dat toegekend kan worden.
Er ontbreekt echter nog 1 element, namelijk de hoeveelheid resources waarover onze node beschikt.
Dit kan gedaan worden in de node-resources.xml of yarn-site.xml.
Elke node moet dit voor zichzelf instellen in zijn eigen configuratiefiles.
Om het aantal cores en Ram-geheugen toe te voegen moet onderstaande xml toegevoegd worden:

    <property>
        <name>yarn.nodemanager.resource-type.vram</name>
        <value>4G</value>
    </property>

Voeg nu als oefening ook een stuk code toe om aan te geven dat je over twee gpu's bezit.

## Aanmaken resource profile

Om het aanvragen van een set van resources te vereenvoudigen en veiliger te maken tegen fouten kunnen er profielen aangemaakt worden om de aangevraagde resources te standaardiseren.
Hiervoor moeten we eerst het gebruik van profiles enablen. Dit kan door het volgende te plaatsen in de yarn-site.xml

	<property>
		<name>yarn.resourcemanager.resource-profiles.enabled</name>
		<value>true</value>
	</property>

Daarna kan bijvoorbeeld onderstaande json gebruikt worden om 4 profiles te maken die een verschillende configuratie vereisen.

    {
        "small": {
            "memory-mb" : 1024,
            "vcores" : 1
        },
        "default" : {
            "memory-mb" : 2048,
            "vcores" : 2
        },
        "large" : {
            "memory-mb": 4096,
            "vcores" : 4
        },
        "compute" : {
            "memory-mb" : 2048,
            "vcores" : 2,
            "gpu" : 1
        }
    }

Deze json moet in een file met naam resource-profiles.json komen.
Er is echter **1 groot nadeel** aan het gebruik van resource profiles verbonden.
Momenteel is het werken met resource profiles voor resources te reserveren enkel gegarandeerd voor de Command Line Interface.

# Map Reduce

Eerst maken we een aparte directory aan voor alles wat we voor deze notebook gaan gebruiken in hdfs. Dit om conflicten of het overschrijven van gegevens te vermijden.


In [1]:
from hdfs import InsecureClient
client = InsecureClient("http://localhost:9870", user='bigdata')
map = 'MapReduce'
input_files = ['input.txt', 'titanic.csv']

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    for f in client.list(map):
        # extra check om ervoor te zorgen dat startdata niet verwijderd wordt
        if not f in input_files:
            client.delete(map + "/" + f, recursive=True)

for f in input_files:
    if client.status(map + "/" + f, strict=False) is None:
        client.upload(map, f)

## Wat is MapReduce

MapReduce is een programmeermodel om eenvoudig distributed data te verwerken.
Het is belangrijk om te realiseren dat de programma's die je hier schrijft een parallel uitgevoerd worden op verschillende stukjes data (De map-fase) om daarna in de reduce-fase tot een finale output teruggebracht te worden.
Er gebeuren 5 stappen bij het uitvoeren van een MapReduce programma
* Bepalen op welke nodes de code uitgevoerd wordt (wordt door YARN gedaan afhankelijk van de locatie van de blocks)
* Uitvoeren van de Map-code (Geschreven door de developer naar eigen wens)
* Shuffle, ouput van de map-fase doorsturen naar andere nodes die de resultaten gaan reduceren (Automatisch)
* uitvoeren van de Reduce-code (Geschreven door de developer naar eigen wens)
* Combineren van de reduce output tot 1 gehele/finale output (Automatisch)

Uit bovenstaand stappenplan is het duidelijk dat er twee zaken moeten geimplementeerd worden bij het schrijven van een MapReduce toepassing.
Echter zullen we eerst een aantal voorbeelden bestuderen met reeds bestaande implementaties om zo meer vertrouwd te geraken met de flow van MapReduce.

## Voorbeelden van bestaande applicaties

Reeds een aantal default MapReduce applications zijn mee geinstalleerd met Hadoop.
De jar die deze toepassingen bevat kan gevonden worden in hadoop/share/hadoop/mapreduce/hadoop-mapreduce-examples-3.3.0.jar.
Wanneer je deze jar uitvoert met onderstaande commando krijg je een lijst met de beschikbare applicaties

In [1]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar

An example program must be given as the first argument.
Valid program names are:
  aggregatewordcount: An Aggregate based map/reduce program that counts the words in the input files.
  aggregatewordhist: An Aggregate based map/reduce program that computes the histogram of the words in the input files.
  bbp: A map/reduce program that uses Bailey-Borwein-Plouffe to compute exact digits of Pi.
  dbcount: An example job that count the pageview counts from a database.
  distbbp: A map/reduce program that uses a BBP-type formula to compute exact bits of Pi.
  grep: A map/reduce program that counts the matches of a regex in the input.
  join: A job that effects a join over sorted, equally partitioned datasets
  multifilewc: A job that counts words from several files.
  pentomino: A map/reduce tile laying program to find solutions to pentomino problems.
  pi: A map/reduce program that estimates Pi using a quasi-Monte Carlo method.
  randomtextwriter: A map/reduce program that writes 10GB of r

In deze notebook gaan we vooral focussen op het typische probleem van wordcount.
Dit is een toepassing dat gaat tellen hoe vaak elk woord voorkomt in een bepaalde tekst.
Om meer informatie over deze toepassing te krijgen kan je gebruik maken van het volgende commando:

In [2]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar wordcount

Usage: wordcount <in> [<in>...] <out>


In de eerste cell hebben we het input.txt bestand geupload. Of dit correct gebeurd is kan je controleren op de [file explorer van het hdfs](http://localhost:9870/explorer.html#/user/bigdata)

Met onderstaande commando kan nu het precompiled word count example uitgevoerd worden

In [8]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar wordcount /user/bigdata/MapReduce/input.txt /user/bigdata/MapReduce/output

2024-02-27 10:45:48,515 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.20.0.4:8032
2024-02-27 10:45:48,703 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.20.0.5:10200
2024-02-27 10:45:49,247 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1709017322874_0001
2024-02-27 10:45:49,725 INFO input.FileInputFormat: Total input files to process : 1
2024-02-27 10:45:49,846 INFO mapreduce.JobSubmitter: number of splits:1
2024-02-27 10:45:50,145 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1709017322874_0001
2024-02-27 10:45:50,145 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-02-27 10:45:50,343 INFO conf.Configuration: resource-types.xml not found
2024-02-27 10:45:50,343 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2024-02-27 10:45:50,837 INFO impl.YarnClientImpl: Submitted application applic

Na het uitvoeren van het wordcount applicatie is er op het HDFS een extra folder toegevoegd met als naam **output**.
De plaats waar de output bewaard wordt is opgegeven in het bovenstaande commando en kan bekeken worden via de file-explorer van het hdfs.
Deze folder bevat de volgende files:
* Een _SUCCESS file dat leeg is. Deze file wordt gebruikt om aan te geven dat de MapReduce applicatie die resulteerde in de output goed afgerond was.
* Een part-r-00000 file dat de output bevat. Indien de output te groot is kan het zijn dat deze file in meerdere files gesplitst wordt. Hier is dit niet het geval en bevat deze file alle ouput. De output van het bovenstaande commando zijn key-value paren waar de keys de verschillende woorden zijn en de values hoeveel keer elk woord voorkomt.

De output van het commando kan dan met onderstaande code uitgelezen worden:

In [9]:
with client.read(map + '/output/part-r-00000') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

!	1
Dit	1
Hello	1
Wordcount	1
World,	1
een	1
file	1
hello	2
het	1
is	1
om	1
te	1
testen	1
voorbeeld	2
world,	2



## Logs/process monitoring

Bovestaande wordcount applicatie wordt uitgevoerd op YARN op de verschillende nodes van de cluster. Dit houdt in dat YARN gebruikt kan worden om de status van de applicatie op te volgen. Hiervoork kan je server naar [de webpagina van Yarn](http://localhost:8088). Na het klikken op de juiste status krijg je de correcte applicatie te zien. Hier kan je ook de logs bekijken door op logs te klikken.

**LET OP: Doordat we in de webbrowser buiten de docker-containers zitten, kunnen we niet de historyserver hostname gebruiken. Als je een DNS-lookup error krijgt, vervang dan de domeinnaam/hostname door localhost.**

## OEFENING: MAPREDUCE 

Probeer nu op basis van bovenstaande reeds bestaande applicaties de gemiddelde lengte van de woorden in de text te bepalen.

In [2]:
!hadoop jar hadoop-mapreduce-examples-3.3.6.jar wordmean /user/bigdata/MapReduce/input.txt /user/bigdata/MapReduce/output_mean

2024-03-05 07:42:06,555 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
2024-03-05 07:42:06,675 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.20.0.3:10200
2024-03-05 07:42:06,861 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1709623304250_0001
2024-03-05 07:42:07,098 INFO input.FileInputFormat: Total input files to process : 1
2024-03-05 07:42:07,183 INFO mapreduce.JobSubmitter: number of splits:1
2024-03-05 07:42:07,275 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1709623304250_0001
2024-03-05 07:42:07,276 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-03-05 07:42:07,402 INFO conf.Configuration: resource-types.xml not found
2024-03-05 07:42:07,402 INFO resource.ResourceUtils: Unable to find 'resource-types.xml'.
2024-03-05 07:42:07,686 INFO impl.YarnClientImpl: Submitted application applic

In [3]:
with client.read(map + '/output_mean/part-r-00000') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

count	18
length	86



De resource manager houdt ook een overzicht bij van de uitgevoerde applicaties, hun status, runtime en eventuele loggings. Na bovenstaande commando's uit te voeren moet je iets gelijkaardigs zien als in de onderstaande screenshot.

![yarn of mapreduce](images/yarn_001.png)

## Zelf implementeren van MapReduce applicaties

Natuurlijk zijn er veel meer zaken mogelijk om te berekenen met map-reduce toepassingen dan de reeds gecompileerde in hadoop.
Zoals eerder aangehaald valt vooral het coderen van de Map- en Reducestap hierbij op de schouders van de developer.
De standaard programmeertaal van MapReduce is Java en dus ook het grootste deel van de documentatie over MapReduce is geschreven met behulp van Java.
Deze programmas moeten dan gecompileerd worden tot een jar dat geupload kan worden naar de overeenkomstige nodes en daar uitgevoerd.
De api overview van hadoop kan je [hier](https://hadoop.apache.org/docs/stable/api/org/apache/hadoop/mapred/package-summary.html) vinden.

De code voor het wordcount example ziet er als volgt uit:

In [10]:
%%file WordCount.java
// dit schrijft onderstaande cell naar een file in de huidige directory
import java.io.IOException;
import java.util.StringTokenizer;

import org.apache.hadoop.conf.Configuration;
import org.apache.hadoop.fs.Path;
import org.apache.hadoop.io.IntWritable;
import org.apache.hadoop.io.Text;
import org.apache.hadoop.mapreduce.Job;
import org.apache.hadoop.mapreduce.Mapper;
import org.apache.hadoop.mapreduce.Reducer;
import org.apache.hadoop.mapreduce.lib.input.FileInputFormat;
import org.apache.hadoop.mapreduce.lib.output.FileOutputFormat;

public class WordCount {

    // Mapper de classes na extends Mapper<> zijn input-key, input-value, output-key, output-value
    public static class TokenizerMapper extends Mapper<Object, Text, Text, IntWritable>{

        private final static IntWritable one = new IntWritable(1);
        private Text word = new Text();

        public void map(Object key, Text value, Context context) throws IOException, InterruptedException {
            // value bevat de tekst in de huidige blok
            // de StringTokenizer maakt het mogelijk om over elk woord te lopen
            StringTokenizer itr = new StringTokenizer(value.toString());
            // while-loop over alle woorden in de blocks op elke node
            while (itr.hasMoreTokens()) {
                // convert the token naar een text waarde
                word.set(itr.nextToken());
                // voeg een entry toe voor elk woord dat gezien wordt (deze worden in de reduce opgeteld)
                context.write(word, one);
            }
        }
    }

    // Reducer: de classes na extends Reducer<> zijn input-key, input-value, output-key, output-value
    public static class IntSumReducer extends Reducer<Text,IntWritable,Text,IntWritable> {
        private IntWritable result = new IntWritable();

        public void reduce(Text key, Iterable<IntWritable> values, Context context) throws IOException, InterruptedException {
            int sum = 0;
            for (IntWritable val : values) {
                sum += val.get();
            }
            result.set(sum);
            context.write(key, result);
        }
    }

    // configure the MapReduce program
    public static void main(String[] args) throws Exception {
        Configuration conf = new Configuration();
        Job job = Job.getInstance(conf, "word count java");
        job.setJarByClass(WordCount.class);
        // configure mapper
        job.setMapperClass(TokenizerMapper.class);
        // configure combiner (soort van reducer die draait op mapping node voor performantie)
        job.setCombinerClass(IntSumReducer.class);
        // configure reducer
        job.setReducerClass(IntSumReducer.class);
        // set output key-value classes
        job.setOutputKeyClass(Text.class);
        job.setOutputValueClass(IntWritable.class);
        // set input file (first argument passed to the program)
        FileInputFormat.addInputPath(job, new Path(args[0]));
        // set output file  (second argument passed to the program)
        FileOutputFormat.setOutputPath(job, new Path(args[1]));
        // In this case, we wait for completion to get the output/logs and not stop the program to early.
        System.exit(job.waitForCompletion(true) ? 0 : 1);
    }
}

Writing WordCount.java


Deze code bevat drie delen:
* De main() functie: verzorgt de configuratie van de uit te voeren taak. Geeft aan wat de Map en Reduce klassen zijn, wat de input is, hoe de output bewaard wordt ,...
* De Map-klasse met de map() functie bevat de code voor de mapping-fase
* De Reduce-klasse met de reduce() functie bevat de code voor de reduce-fase

De laatste twee klassen zijn hier gecodeerd als geneste klassen. Deze hadden ook in aparte files geplaatst kunnen worden.
Nu moet deze code eerst omgezet/gecompileerd worden tot een .jar file. Deze kan dan analoog uitgevoerd worden als hierboven met het voorbeeldcode.
Deze twee stappen kunnen uitgevoerd worden door onderstaande commando's.

In [11]:
# compileren tot .jar
!javac -cp "hadoop-common-3.3.6.jar:hadoop-mapreduce-client-core-3.3.6.jar" -d . WordCount.java
!jar cvf wordcounter.jar *.class

added manifest
adding: WordCount$IntSumReducer.class(in = 1739) (out= 739)(deflated 57%)
adding: WordCount$TokenizerMapper.class(in = 1736) (out= 754)(deflated 56%)
adding: WordCount.class(in = 1496) (out= 818)(deflated 45%)


In [12]:
# execute
!hadoop jar wordcounter.jar WordCount /user/bigdata/MapReduce/input.txt /user/bigdata/MapReduce/output_java

2024-02-27 10:56:04,868 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at resourcemanager/172.20.0.4:8032
2024-02-27 10:56:04,977 INFO client.AHSProxy: Connecting to Application History server at historyserver/172.20.0.5:10200
2024-02-27 10:56:05,116 WARN mapreduce.JobResourceUploader: Hadoop command-line option parsing not performed. Implement the Tool interface and execute your application with ToolRunner to remedy this.
2024-02-27 10:56:05,128 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1709017322874_0002
2024-02-27 10:56:05,382 INFO input.FileInputFormat: Total input files to process : 1
2024-02-27 10:56:05,475 INFO mapreduce.JobSubmitter: number of splits:1
2024-02-27 10:56:05,568 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1709017322874_0002
2024-02-27 10:56:05,568 INFO mapreduce.JobSubmitter: Executing with tokens: []
2024-02-27 10:56:05,719 INFO conf.Configuratio

In [13]:
with client.read(map + '/output_java/part-r-00000') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

!	1
Dit	1
Hello	1
Wordcount	1
World,	1
een	1
file	1
hello	2
het	1
is	1
om	1
te	1
testen	1
voorbeeld	2
world,	2



## Implementeren via Python

Hoewel het Hadoop Ecosysteem geprogrammeerd is in Java en het dus er goed mee samenwerkt, is het niet verplicht om Java te gebruiken om MapReduce applicaties te schrijven.
Java krijgt namelijk veel kritiek, vooral doordat er veel code nodig is om eenvoudige zaken te programmeren.
Om andere programmeertalen te gebruiken worden er verscheidene API's aangeboden door Hadoop, namelijk
* Hadoop Streaming
    * Communicatie via stdin/stdout
    * Gebruikt door hadoopy, mrjob, ...
* Hadoop Pipes
    * C++ interface voor Hadoop
    * Communicatie via sockets
    * Gebruikt door pydoop


### Mrjob

De eerste API die we bekijken is MrJobs.
Deze API maakt gebruik van de Hadoop Streaming API.
De voordelen van MrJobs zijn:
* Uitgebreidde documentatie
* Code kan lokaal uitgevoerd worden als test
* Data serialisatie gebeurt automatisch (nadeel van Streaming API)
* Werkt met Amazon Elastic MapReduce en Google Cloud Dataproc

Het grootste nadeel is dat de StreamingAPI niet de volledige kracht heeft van het Hadoop ecosysteem omdat alles omgezet wordt naar strings (jsons)

Installatie van deze package zou reeds gebeurd moeten zijn, anders kan je het doen met behulp van het volgende commando:

In [ ]:
!echo bigdata | sudo -S pip install mrjob

In [2]:
%%file wordcount_mrjob.py
# dit moet altijd in een python file zitten, %%file moet altijd vanboven staan
# mrjob - wordcount example
from mrjob.job import MRJob

class MRWordCount(MRJob):
    def mapper(self, _, line):
        for word in line.split():
            yield (word, 1)

    def reducer(self, word, counts):
        yield (word, sum(counts))

if __name__ == '__main__':
    MRWordCount.run()

Overwriting wordcount_mrjob.py


Deze code kan uitgevoerd worden door het commando hieronder.
Er zijn twee belangrijke parameters om toe te voegen aan het commando.
* -r: Deze parameter geeft aan welke file uitgelezen wordt. Dit kan een lokale file zijn of een file op het hdfs. Om een hdfs-file aan te spreken gebruik je de volgende structuur: hdfs:///{path to file}
* -o: Deze parameter geeft aan waar de data moet bewaard worden. Als deze parameter ontbreekt wordt het in de stdout geprint. Met de parameter kan je aangeven in welke file (lokaal of hdfs) de output moet bewaard worden.

Meer informatie over hoe mrjob applicaties gestart kunnen worden vind je in [de documentatie](https://mrjob.readthedocs.io/en/latest/guides/quickstart.html#running-your-job-different-ways)

In [4]:
!python3 wordcount_mrjob.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt -o hdfs:///user/bigdata/MapReduce/output.txt

# Probleem bij week 2 was dat er een spatie stond in de foldernaam

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/wordcount_mrjob.root.20240305.074936.029187
uploading working dir files to hdfs:///user/root/tmp/mrjob/wordcount_mrjob.root.20240305.074936.029187/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/wordcount_mrjob.root.20240305.074936.029187/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar1886861388484282530/] [] /tmp/streamjob6148184741574442559.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
  Connecting to Application History server at historyserver/172.20.0.3:10200
  Connecting to ResourceManager at resourcemanager/

In [5]:
with client.read(map + '/output.txt/part-00000') as reader:
    content = reader.read()
    print(content.decode('utf-8'))

"!"	1
"Dit"	1
"Hello"	1
"Wordcount"	1
"World,"	1
"een"	1
"file"	1
"hello"	2
"het"	1
"is"	1
"om"	1
"te"	1
"testen"	1
"voorbeeld"	2
"world,"	2



De bovenstaande manier vereist echter dat je steeds een python-file aanmaakt die dan via commandline gestart wordt.
De reden hiervoor is is dat **de file naar de cluster verstuurd wordt**.

## Oefeningen

Maak de nodige mapreduce applicaties voor de volgende zaken te berekenen (je moet de output niet bewaren in een file):
* De gemiddelde woordlengte
* Het aantal keer dat elk karakter voorkomt
* Het aantal woorden dat begint met elke letter
* Het aantal woorden in de tekst

Als een applicatie crasht, kan het zijn dat ze op de cluster nog een hele tijd actief blijft en het starten van nieuwe applicaties kan tegenhouden.
Met onderstaande commando kan je een bestande applicatie afsluiten/killen

In [ ]:
# killen van een applicatie
yarn application -kill <Application_ID>
# appliation id kan je vinden via de webbrowser van yarn (localhost:8088)

In [12]:
%%file vraag1.py
# vraag 1: gemiddelde woordlengte
from mrjob.job import MRJob

class Vraag1(MRJob):
    def mapper(self, _, line):
        for word in line.split():
            yield ("average length", len(word)) # spatie kan gebruikt worden om een unieke key te hebben als je ook iets doet woord per woord

    def reducer(self, word, counts):
        # zelf itereren
        # Calculate the average word length
        #total_length = 0
        #total_count = 0
        #for value in values:
        #    total_length += value
        #    total_count += 1
        #average_length = total_length / total_count
        #yield (key, average_length)

        # dit gaat niet werken want counts is een generator object
        # wijst naar het eerste object, en dat heeft een functie next()
        # je kan niet terug naar het begin gaan, je kan er maar 1 keer over itereren
        #counts = list(counts)
        mean = sum(counts) / len(counts)
        print('mean', mean)
        
        yield (word, mean)

if __name__ == '__main__':
    Vraag1.run()

Overwriting vraag1.py


In [13]:
# test vraag 1
!python3 vraag1.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/vraag1.root.20240305.085049.802176
uploading working dir files to hdfs:///user/root/tmp/mrjob/vraag1.root.20240305.085049.802176/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/vraag1.root.20240305.085049.802176/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar1898398899789495779/] [] /tmp/streamjob5905063239708396912.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
  Connecting to Application History server at historyserver/172.20.0.3:10200
  Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
  Connectin

In [ ]:
# andere manier om de log files te bekijken van de applicatie:
#!yarn logs -applicationId application_1709623304250_0004
# yarn logs -containerId container_e34_1709623304250_0004_01_000007
# https://hadoop.apache.org/docs/stable/hadoop-yarn/hadoop-yarn-site/YarnCommands.html

In [14]:
%%file vraag2.py
# vraag 2: het aantal keer dat elk karakter voorkomt

from mrjob.job import MRJob

class Vraag2(MRJob):
    def mapper(self, _, line):
        for c in line:
            if c.isalpha():
                yield (c.lower(), 1)

    def reducer(self, word, counts):
        yield (word, sum(counts))

if __name__ == '__main__':
    Vraag2.run()

Writing vraag2.py


In [15]:
# test vraag 2
!python3 vraag2.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/vraag2.root.20240305.090005.269459
uploading working dir files to hdfs:///user/root/tmp/mrjob/vraag2.root.20240305.090005.269459/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/vraag2.root.20240305.090005.269459/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar5547305628161377497/] [] /tmp/streamjob3902315544997601875.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
  Connecting to Application History server at historyserver/172.20.0.3:10200
  Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
  Connectin

In [16]:
%%file vraag3.py
# vraag 3: * Het aantal woorden dat begint met elke letter

from mrjob.job import MRJob

class Vraag3(MRJob):
    def mapper(self, _, line):
        for word in line.split():
            if len(word) > 0 and word[0].isalpha():
                yield (word[0].lower(), 1)

    def reducer(self, word, counts):
        yield (word, sum(counts))

if __name__ == '__main__':
    Vraag3.run()

Writing vraag3.py


In [17]:
# test vraag 3
!python3 vraag3.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/vraag3.root.20240305.090201.251870
uploading working dir files to hdfs:///user/root/tmp/mrjob/vraag3.root.20240305.090201.251870/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/vraag3.root.20240305.090201.251870/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar162690889028183778/] [] /tmp/streamjob5655895214953926387.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
  Connecting to Application History server at historyserver/172.20.0.3:10200
  Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
  Connecting

In [18]:
%%file vraag4.py
# vraag 4: Het aantal woorden in de tekst

from mrjob.job import MRJob

class Vraag4(MRJob):
    def mapper(self, _, line):
        for word in line.split():
            yield ("aantal woorden", 1)

    def reducer(self, word, counts):
        yield (word, sum(counts))

if __name__ == '__main__':
    Vraag4.run()

Writing vraag4.py


In [19]:
# test vraag 4
!python3 vraag4.py -r hadoop hdfs:///user/bigdata/MapReduce/input.txt

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /opt/hadoop-3.3.6/bin...
Found hadoop binary: /opt/hadoop-3.3.6/bin/hadoop
Using Hadoop version 3.3.6
Looking for Hadoop streaming jar in /opt/hadoop-3.3.6...
Found Hadoop streaming jar: /opt/hadoop-3.3.6/share/hadoop/tools/lib/hadoop-streaming-3.3.6.jar
Creating temp directory /tmp/vraag4.root.20240305.090308.833739
uploading working dir files to hdfs:///user/root/tmp/mrjob/vraag4.root.20240305.090308.833739/files/wd...
Copying other local files to hdfs:///user/root/tmp/mrjob/vraag4.root.20240305.090308.833739/files/
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar8694725971371442727/] [] /tmp/streamjob4036326450548955324.jar tmpDir=null
  Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
  Connecting to Application History server at historyserver/172.20.0.3:10200
  Connecting to ResourceManager at resourcemanager/172.20.0.9:8032
  Connectin

## Werken met MapReduce voor gestructureerde data

Mapreduce kan ook gebruikt worden om te werken met gestructureerde data (bijvoorbeeld een csv) waar elke rij 1 data-element voorsteld.
Het is hier echter wel belangrijk dat alle data op 1 lijn gecombineerd wordt dus multiline csv's, jsons of xml bestanden kunnen niet verwerkt worden.

Onderstaande code is een voorbeeld voor hoe je een csv kan uitlezen en een aantal statistieken kan berekenen. Hierin leer je vooral:
* Hoe de csv lijn per lijn te verwerken en kolommen te detecteren
* Hoe de header rij eruit filteren
* Hoe meerdere berekeningen op een iterator te doen

We gebruiken hiervoor de titanic.csv file. Let op dat die naam van de passagier hierbij komma's kan bevatten dus dit vereist wat extra aandacht.
Daarnaast gebruiken we de lokale versie van het bestand en niet de geuploadde versie.

We willen de volgende zaken berekenen door middel van 1 map-reduce applicatie:
* Gemiddelde leeftijd
* Percentage overleeft
* Percentage mannelijke passagiers
* Percentage vrouwelijke passagiers die het overleefd hebben: 30% betekend dat 30% van de vrouwelijke passagiers het overleefd hebben

In [29]:
%%file structured.py
from mrjob.job import MRJob
import csv
from io import StringIO

col_survived = 1
col_gender = 4
col_age = 5

class Structured(MRJob):
    def mapper(self, _, line):

        # skip de header rij
        if line == "PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked":
            return

        # converteer lijn naar rij van column values
        csv_file = StringIO(line)
        cols = next(csv.reader(csv_file))  # csv.reader returned een generator die over de lijnen itereert

        if cols[col_age] != '':
            yield ("leeftijd", float(cols[col_age]))

        if cols[col_survived] != '':
            yield('overleefd', float(cols[col_survived]))

        if cols[col_gender] != '':
            yield('perc_man', int(cols[col_gender] == 'male'))

        if cols[col_gender] != '' and cols[col_survived] != '':
            if cols[col_gender] == 'female':
                yield('vrouw_overleefd', float(cols[col_survived]))

    def reducer(self, word, counts):

        if word == 'leeftijd':
            counts = list(counts)
            yield(word, sum(counts)/len(counts))
        elif word == 'overleefd' or word =='perc_man' or word == 'vrouw_overleefd':
            counts = list(counts)
            yield(word, sum(counts)/len(counts) * 100)         

if __name__ == '__main__':
    Structured.run()

Overwriting structured.py


In [30]:
# test structured
!python3 structured.py titanic.csv

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/structured.root.20240305.095045.612705
Running step 1 of 1...
job output is in /tmp/structured.root.20240305.095045.612705/output
Streaming final output from /tmp/structured.root.20240305.095045.612705/output...
"leeftijd"	29.69911764705882
"perc_man"	64.75869809203144
"vrouw_overleefd"	74.20382165605095
"overleefd"	38.38383838383838
Removing temp directory /tmp/structured.root.20240305.095045.612705...


## Afsluitende opmerkingen

Het is belangrijk om te beseffen dat deze applicatie op meerdere nodes kan tegelijkertijd uitgevoerd worden. 
Dit heeft als gevolg dat je geen persistente counters kan toevoegen in de reducer om gemiddelden en dergelijke te berekenen.
Een globaal overzicht van het dataframe kan maar **in de mapper** behaald worden.

Daarnaast is er nog een andere stap mogelijk dan mapper of reducer. Dat is **de combiner** stap. Dit is een stap die runt per node en gebruikt kan worden om al wat combinaties te doen zodat er minder data tussen nodes moet verstuurd worden. In grote applicaties/datasets kan dit heel wat internettrafiek besparen wat de werking van de cluster ten goede komt.

Ten derde is het ook mogelijk om meerdere stappen te definieren in de mrjob applicatie. Dit kan door de steps functie te implementeren en daar elke stap in te definieren. Elke stap hierin kan beschikken over een eigen mapper/reducer en combiner functie. Meer informatie vind je [in de documentatie](https://mrjob.readthedocs.io/en/latest/guides/quickstart.html#writing-your-second-job)

## Verdere oefeningen

Gebruik nu de titanic csv om de volgende zaken te berekenen:
* Het aantal unieke plaatsen waar personen aan boord zijn gekomen (embarked kolom)
* Het aantal ontbrekende waarden in de Cabin kolom
* De volgende statistische waarden voor de ticketprijs (Fare) kolom: min, max, mean, std
* De langste naam van een passagier
* Het aantal passagiers per klasse
* Het totaal aantal passagiers op de titanic